# Understand feature table summary

In [1]:
import pandas as pd

metaData = pd.read_csv(
    '/home/truongphi/Desktop/meta/QIIME/FMT/work/0d/5f61f99d20e960af787b108e74bba1/sample_metadata_filtered.tsv',
    sep="\t")
print(metaData.head())

                              sample-id sequencing-run  sample-type  \
0                             #q2:types    categorical  categorical   
1  595e93ed.e4b3.42b5.91b1.1ad4f31e7134          run-1        stool   
2  bd64992d.87cc.464f.9e1b.9c0156ed65d8          run-1        stool   
3  e7a2370c.f80c.4703.b2ee.10c65967e05f          run-1        stool   
4  11f3f37a.3698.4031.8183.75f6f2c6c04c          run-1        stool   

  treatment-group   subject-id     week     gsrs gsrs-diff  \
0     categorical  categorical  numeric  numeric   numeric   
1       treatment         B101        0      2.9         0   
2       treatment         B101        3      1.8      -1.1   
3       treatment         B101       10      1.3      -1.6   
4       treatment         B101       18      1.3      -1.6   

  administration-route      age       gender   weight   height      bmi  
0          categorical  numeric  categorical  numeric  numeric  numeric  
1                 Oral     12.5            m       6

In [ ]:
from biom import load_table
table = load_table('feature_table/feature-table.biom') 
print(dir(table))

In [ ]:
dataframe = pd.DataFrame(table.to_dataframe())
dataframe.to_csv('feature_table/data.csv')

In [ ]:
table_data = pd.read_csv('feature_table/data.csv', index_col=0)
table_data

## Frequency per sample

In [ ]:
def countFeature(lista):
    count = 0
    for ele in lista:
        if ele > 0:
            count += 1
    return count

FixList = lambda lista: [i for l in lista for i in l]

CountData = lambda features, table_data: [countFeature(table_data[table_data.index==feature].transpose().astype('float64').values) for feature in features]
SumData = lambda features, table_data: [sum(FixList(table_data[table_data.index==feature].transpose().astype('float64').values)) for feature in features]

def MakeTable(features, table_data):
    count_values = CountData(features, table_data)
    sum_values = SumData(features, table_data)

    frePerFeature = pd.DataFrame(zip(sum_values, count_values), index=features, columns=['Frequency', 'Count'])
    # print(frePerFeature)
    return frePerFeature

In [ ]:
headers = table_data.columns
sum_values = [sum(table_data[header]) for header in headers]
count_values = [countFeature(table_data[header].astype('float64').values) for header in headers]

frePerSample = pd.DataFrame(zip(sum_values, count_values), index=headers, columns=['Frequency', 'Count'])
print(frePerSample)

In [ ]:
print(frePerSample.describe())
print(frePerSample.median())

In [ ]:
import seaborn as sns
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

sns.histplot(data=frePerSample["Frequency"], bins=8)
plt.show()

## Frequency per Features

In [ ]:
features = table_data.index

frePerFeature = MakeTable(features, table_data)
print(frePerFeature)

In [ ]:
print(frePerFeature.describe())
print(frePerFeature.median())

## Interactive

In [ ]:
sample_depth = 1206

retain_sample = frePerSample[frePerSample['Frequency']>=sample_depth]
num_retain_sample = len(retain_sample)
print(f'The number of samples remain: {num_retain_sample}')
print(f'The number of samples remain: {sample_depth*num_retain_sample}')


In [ ]:
num_sample = len(frePerSample)
# print(num_sample)

num_feature = len(frePerFeature)
# print(num_feature)

max_depth = max(frePerSample['Frequency'])
# print(max_depth)

total_fre = sum(frePerSample['Frequency'])

In [ ]:
def SampleDepth(sample_depth, frePerSample):
    retain_sample = frePerSample[frePerSample['Frequency']>=sample_depth]
    num_retain_sample = len(retain_sample)*100/num_sample
    num_retain_feature = sample_depth*num_retain_sample*100/total_fre
    return num_retain_feature, num_retain_sample

In [ ]:
import numpy as np

depths = list(range(int(max_depth)+1))

retain_feature = [] 
retain_sample = []
for depth in depths:
    num1, num2 = SampleDepth(depth, frePerSample)
    # print(num1, num2)
    retain_feature.append(num1)
    retain_sample.append(num2)

In [ ]:
import matplotlib.pyplot as plt

# fig, ax1 = plt.subplots(figsize=(15, 8))
# ax2 = ax1.twinx()

# l1, = ax1.plot(depths, retain_feature, label='Number of features')
# l2, = ax2.plot(depths, retain_sample, color='y', label='Number of samples')
# ax1.set_xlabel("Sampling Depth")
# ax1.set_ylabel("Number of features")
# ax2.set_ylabel("Number of samples")
# plt.xticks(list(range(0, int(max_depth), 1000)))
# plt.legend([l1, l2], ['Number of features', 'Number of samples'])
# plt.show();

In [ ]:

plt.figure(figsize=(15,8))
plt.plot(depths, retain_feature, label='Percent of features');
plt.plot(depths, retain_sample, color='y', label='Percent of samples');
plt.xlabel('Sample Depth')
plt.xticks(list(range(0,int(max_depth)+1,1000)))
plt.ylabel('Percentage')
plt.ylim([0,100])
plt.xlim([0,int(max_depth)+1])
plt.legend()
plt.show()
# plt.tight_layout()
# plt.savefig('graph.png', dpi=300)

# Rarefraction

In [4]:
import pandas as pd

rare_data = pd.read_csv('rarefraction/faith_pd.csv')
rare_data.head()

,sample-id,depth-1_iter-1,depth-1_iter-2,depth-1_iter-3,depth-1_iter-4,depth-1_iter-5,depth-1_iter-6,depth-1_iter-7,depth-1_iter-8,depth-1_iter-9,...,depth-6000_iter-7,depth-6000_iter-8,depth-6000_iter-9,depth-6000_iter-10,sequencing-run,sample-type,treatment-group,subject-id,administration-route,gender
0,0bb5e24a.aa34.48e7.b1f9.e761ac2dc6b4,1.278692,1.587817,1.610428,1.271537,1.278692,1.278692,1.278692,1.278692,1.494667,...,10.652861,10.652861,10.652861,10.652861,run-1,swab,treatment,B101,Oral,m
1,101c02ce.4f8d.4394.83a3.0c406831d934,1.587817,1.577563,1.595031,1.585158,1.627485,1.278692,1.595147,1.354253,1.506647,...,9.836916,9.836916,9.836916,9.836916,run-1,swab,treatment,B103,Rectal,f
2,104e5902.1c3e.417f.bb22.124bfca61a61,1.284688,1.278692,1.418343,1.610428,1.295372,1.594654,1.278692,1.587586,1.271537,...,12.274065,12.274065,12.274065,12.274065,run-1,swab,treatment,B101,Oral,m
3,11c4be47.fdce.4135.8acf.0894f2da5ede,1.271147,1.271147,1.300423,1.271147,1.595031,1.271147,1.271147,1.271147,1.271147,...,NaN,NaN,NaN,NaN,run-1,swab,treatment,B105,Oral,m
4,11c70d46.e0a4.4797.ad7b.3677fe93d73b,1.353522,1.353522,1.353522,1.602882,1.602882,1.353522,1.602882,1.353522,1.278692,...,NaN,NaN,NaN,NaN,run-1,swab,treatment,B105,Oral,m
